## random.shuffle 기능을 이용하여 
## training data 와 test data 분리 하는 DataGeneration class 구현

#### 수치미분 함수 / 시그모이드 함수 선언

In [1]:
import numpy as np
import random
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

#### ThoracicSugery class 구현

In [2]:
# ThoracicSugery Class

class ThoracicSugery:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, name, input_nodes, hidden1_nodes, hidden2_nodes, output_nodes, learning_rate):
        
        self.name = name
        
        # 2층 hidden layer unit 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden1_nodes)  
        self.b2 = np.random.rand(hidden1_nodes)
        
        # 3층 hidden layer unit  
        self.W3 = np.random.rand(hidden1_nodes, hidden2_nodes)
        self.b3 = np.random.rand(hidden2_nodes)
        
        # 4층 output layer unit  
        self.W4 = np.random.rand(hidden2_nodes, output_nodes)
        self.b4 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        print(self.name, " is created !!!")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, input_data):    
        
        z2 = np.dot(input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    def accuracy(self, input_data, target_data):
        
        matched_list = []
        not_matched_list = []
        
        # list which contains (index, label, prediction) value
        index_label_prediction_list = []
        
        # temp list which contains label and prediction in sequence
        temp_list = []
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
                temp_list.append(index)
                temp_list.append(target_data[index])
                temp_list.append(logical_val)
                
                index_label_prediction_list.append(temp_list)
                
                temp_list = []
                
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list, index_label_prediction_list
    
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
        self.W4 -= self.learning_rate * numerical_derivative(f, self.W4)
    
        self.b4 -= self.learning_rate * numerical_derivative(f, self.b4)

#### DataGeneration class 구현

In [3]:
class DataGeneration:
    
    def __init__(self, file_path, seperation_rate, is_normalized=False):
        
        self.file_path = file_path
        
        self.seperation_rate = seperation_rate
        
        self.is_normalized = is_normalized
        
        print("DataGeneration object is created !!")
    
    # np.max() 를 이용한 데이터 정규화
    def data_normalize_using_max(self, loaded_data):
        
        # 각 열의 최대값을 찾기 위해 행과 열을 바꾸어 줌. 즉 전치향렬을 만들어줌
        transpose_loaded_data = loaded_data.T        

        print("transpose_loaded_data.shape = ", transpose_loaded_data.shape)
        
        # 전치행렬을 위한 리스트
        transpose_normalize_data_list = []

        for index in range(len(transpose_loaded_data)):
    
            max_value = np.max(transpose_loaded_data[index, :])   # 각 행의 최대값을 찾음
    
            # 최대값이 1 이상이면 최대값으로 나누어서 
            # 모든 데이터가 0 ~ 1 사이에 오도록 함
            if max_value > 1.0:  
        
                transpose_normalize_data_list.append(transpose_loaded_data[index, :] / max_value)
       
            # 최대값이 1 보다 작으면 해당 값을 그대로 사용함. 
            ## 왜냐하면 1보다 작은 값이면 굳이 바꿀 필요가 없음
            else:      
        
                transpose_normalize_data_list.append(transpose_loaded_data[index, :])
        

        # 리스트를 numpy type 으로 변환
        transpose_normalize_data = np.array(transpose_normalize_data_list)

        print(transpose_normalize_data.shape)

        # 데이터 저장을 위해 다시 전치행렬을 통해 행과 열을 바꿈
        normalize_data = transpose_normalize_data.T

        print(normalize_data.shape)

        # normalize 파일저장
        np.savetxt('./Normalize_Diabetes_data.csv', normalize_data, delimiter=',')
        
        return normalize_data

    # shuffle 기능을 이용하여 training_data / test_data 생성
    def generate(self):
    
        # 데이터 불러오기

        loaded_data = np.loadtxt(self.file_path, delimiter=',', dtype=np.float32)
        
        print("[debug. before data normalization] loaded_data[0] = ", loaded_data[0])

        print("loaded_data.shape = ", loaded_data.shape)
        
        if (self.is_normalized == True):
            
            loaded_data = self.data_normalize_using_max(loaded_data)
            
            print("[debug. after data normalization] loaded_data[0] = ", loaded_data[0])

        # 임시 저장 리스트
        training_data_list = []
        test_data_list = []

        # 분리비율에 맞게 테스트데이터로 분리
        total_data_num = len(loaded_data)
        test_data_num = int(len(loaded_data) * self.seperation_rate)

        print("total_data_num = ", total_data_num, ", test_data_num = ", test_data_num)

        # 전체 데이터 인덱스를 가지고 있는 리스트 생성
        total_data_index_list = [ index for index in range(total_data_num) ]

        # random.shuffle 을 이용하여 인덱스 리스트 생성
        random.shuffle(total_data_index_list)  # 전체 인덱스가 랜덤하게 섞여진 리스트로 변형된다

        # test data 를 위한 인덱스는 total_data_index_list 로뷰터 앞에서 40 % 의 데이터 인덱스
        test_data_index_list = total_data_index_list[ 0:test_data_num ]

        print("length of test_data_index_list = ", len(test_data_index_list))

        # training data 를 위한 인덱스는 total_data_index_list 에서 test data 인덱스를 제외한 나머지 부분
        training_data_index_list = total_data_index_list[ test_data_num: ]

        print("length of training_data_index_list = ", len(training_data_index_list))

        # training data 구성
        for training_data_index in training_data_index_list:
    
            training_data_list.append(loaded_data[training_data_index])

        # test data 구성
        for test_data_index in test_data_index_list:
    
            test_data_list.append(loaded_data[test_data_index])

        # generate training data from training_data_list using np.arrya(...)
        training_data = np.array(training_data_list)

        # generate test data from test_data_list using np.arrya(...)
        test_data = np.array(test_data_list)

        # verification shape
        print("training_data.shape = ", training_data.shape)
        print("test_data.shape = ", test_data.shape)

        # save training & test data (.csv)
        np.savetxt('./random_ThoracicSugery_training_data.csv', training_data, delimiter=',')
        np.savetxt('./random_ThoracicSugery_test_data.csv', test_data, delimiter=',')
        
        return training_data, test_data

#### 정규화하지 않은 DataGeneration 객체 / ThoracicSugery 객체 생성 및 학습

In [10]:
# DataGeneration 객체 생성. normalization 없음
seperation_rate = 0.3
data_obj = DataGeneration('./ThoracicSurgery.csv', seperation_rate)
(training_data, test_data) = data_obj.generate()


#hyper-parameter
i_nodes = training_data.shape[1] - 1    # input nodes 개수
h1_nodes = 2  # hidden 1 nodes 개수. 
h2_nodes = 2  # hidden 2 nodes 개수
o_nodes = 1    # output nodes 개수
lr = 1e-2      # learning rate. hi_node = 2, 1e-1 에서 수렴
epochs = 20   # 반복횟수. 


# ThoracicSugery 객체 생성
ThoracicSugery_obj = ThoracicSugery("ThoracicSurgery", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(training_data)):
        
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        ThoracicSugery_obj.train(input_data, target_data)
        
    print("epochs = ", step, "loss value = ", ThoracicSugery_obj.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

DataGeneration object is created !!
[debug. before data normalization] loaded_data[0] =  [293.    1.    3.8   2.8   0.    0.    0.    0.    0.    0.   12.    0.
   0.    0.    1.    0.   62.    0. ]
loaded_data.shape =  (470, 18)
total_data_num =  470 , test_data_num =  141
length of test_data_index_list =  141
length of training_data_index_list =  329
training_data.shape =  (329, 18)
test_data.shape =  (141, 18)
ThoracicSurgery  is created !!!
Neural Network Learning using Numerical Derivative...
epochs =  0 loss value =  0.3193476719592501
epochs =  1 loss value =  0.19869056971404522
epochs =  2 loss value =  0.17262912823416812
epochs =  3 loss value =  0.16477731318844077
epochs =  4 loss value =  0.16216486153994847
epochs =  5 loss value =  0.1612665414379369
epochs =  6 loss value =  0.16095409521422957
epochs =  7 loss value =  0.16084496436527526
epochs =  8 loss value =  0.16080676890933393
epochs =  9 loss value =  0.16079337003102676
epochs =  10 loss value =  0.1607886454

#### test_data를 이용하여 정확도 검증

In [11]:
test_input_data = test_data[ :, 0:-1 ]
test_target_data = test_data[ :, -1 ]

(true_list_1, false_list_1, index_label_prediction_list) = ThoracicSugery_obj.accuracy(test_input_data, test_target_data) 

Accuracy =>  0.851063829787234


In [12]:
print(index_label_prediction_list)

[[5, 1.0, 0], [6, 1.0, 0], [7, 1.0, 0], [9, 1.0, 0], [20, 1.0, 0], [26, 1.0, 0], [37, 1.0, 0], [38, 1.0, 0], [40, 1.0, 0], [44, 1.0, 0], [61, 1.0, 0], [75, 1.0, 0], [83, 1.0, 0], [96, 1.0, 0], [102, 1.0, 0], [109, 1.0, 0], [123, 1.0, 0], [128, 1.0, 0], [130, 1.0, 0], [131, 1.0, 0], [139, 1.0, 0]]


#### 정규화된  DataGeneration 객체 / ThoracicSugery 객체 생성 및 학습

In [16]:
# 정규화된 DataGeneration 객체 생성. 
seperation_rate = 0.3
data_obj = DataGeneration('./ThoracicSurgery.csv', seperation_rate, True)
(training_data, test_data) = data_obj.generate()


#hyper-parameter
i_nodes = training_data.shape[1] - 1    # input nodes 개수
h1_nodes = 2  # hidden 1 nodes 개수. 
h2_nodes = 2  # hidden 2 nodes 개수
o_nodes = 1    # output nodes 개수
lr = 1e-3      # learning rate. hi_node = 2, 1e-1 에서 수렴
epochs = 20   # 반복횟수. 


# ThoracicSugery 객체 생성
ThoracicSugery_obj = ThoracicSugery("ThoracicSurgery", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

print("Neural Network Learning using Numerical Derivative...")

start_time = datetime.now()

for step in range(epochs):
    
    for index in range(len(training_data)):
        
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        ThoracicSugery_obj.train(input_data, target_data)
        
    print("epochs = ", step, "loss value = ", ThoracicSugery_obj.loss_val())

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

DataGeneration object is created !!
[debug. before data normalization] loaded_data[0] =  [293.    1.    3.8   2.8   0.    0.    0.    0.    0.    0.   12.    0.
   0.    0.    1.    0.   62.    0. ]
loaded_data.shape =  (470, 18)
transpose_loaded_data.shape =  (18, 470)
(18, 470)
(470, 18)
[debug. after data normalization] loaded_data[0] =  [0.62340426 0.125      0.60317457 0.03244496 0.         0.
 0.         0.         0.         0.         0.85714287 0.
 0.         0.         1.         0.         0.7126437  0.        ]
total_data_num =  470 , test_data_num =  141
length of test_data_index_list =  141
length of training_data_index_list =  329
training_data.shape =  (329, 18)
test_data.shape =  (141, 18)
ThoracicSurgery  is created !!!
Neural Network Learning using Numerical Derivative...
epochs =  0 loss value =  1.0059449446451982
epochs =  1 loss value =  0.8186998204728014
epochs =  2 loss value =  0.6813147189976214
epochs =  3 loss value =  0.579869792522232
epochs =  4 loss va

In [17]:
test_input_data = test_data[ :, 0:-1 ]
test_target_data = test_data[ :, -1 ]

(true_list_1, false_list_1, index_label_prediction_list) = ThoracicSugery_obj.accuracy(test_input_data, test_target_data) 

Accuracy =>  0.8865248226950354


In [18]:
print(index_label_prediction_list)

[[4, 1.0, 0], [6, 1.0, 0], [16, 1.0, 0], [30, 1.0, 0], [31, 1.0, 0], [34, 1.0, 0], [39, 1.0, 0], [69, 1.0, 0], [72, 1.0, 0], [79, 1.0, 0], [81, 1.0, 0], [86, 1.0, 0], [110, 1.0, 0], [114, 1.0, 0], [130, 1.0, 0], [131, 1.0, 0]]
